In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pima-indians-diabetes-database/diabetes.csv


In [2]:
filepath="../input/pima-indians-diabetes-database/diabetes.csv"
diab_data=pd.read_csv(filepath)

In [3]:
diab_data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
y = diab_data.Outcome #setting prediction target


In [5]:
diab_data.dtypes

Pregnancies                   int64
Glucose                       int64
BloodPressure                 int64
SkinThickness                 int64
Insulin                       int64
BMI                         float64
DiabetesPedigreeFunction    float64
Age                           int64
Outcome                       int64
dtype: object

In [6]:
diab_data.corr() #data seems to have 46% correlation with glucose level
                 #did they take glucose level before or after knowing the diabetic condition?

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
Pregnancies,1.000000,0.129459,0.141282,-0.081672,-0.073535,0.017683,-0.033523,0.544341,0.221898
Glucose,0.129459,1.000000,0.152590,0.057328,0.331357,0.221071,0.137337,0.263514,0.466581
BloodPressure,0.141282,0.152590,1.000000,0.207371,0.088933,0.281805,0.041265,0.239528,0.065068
SkinThickness,-0.081672,0.057328,0.207371,1.000000,0.436783,0.392573,0.183928,-0.113970,0.074752
Insulin,-0.073535,0.331357,0.088933,0.436783,1.000000,0.197859,0.185071,-0.042163,0.130548
BMI,0.017683,0.221071,0.281805,0.392573,0.197859,1.000000,0.140647,0.036242,0.292695
DiabetesPedigreeFunction,-0.033523,0.137337,0.041265,0.183928,0.185071,0.140647,1.000000,0.033561,0.173844
Age,0.544341,0.263514,0.239528,-0.113970,-0.042163,0.036242,0.033561,1.000000,0.238356
Outcome,0.221898,0.466581,0.065068,0.074752,0.130548,0.292695,0.173844,0.238356,1.000000


In [7]:
#A Function to check for missing data
def data_missing_percent(data):
    total_cells            = data.size
    missing_values_count   = data.isnull().sum()
    total_missing_values   = missing_values_count.sum()
    percent_missing_values = (total_missing_values/total_cells)*100
    print(percent_missing_values,'%')

In [8]:
data_missing_percent(diab_data) #using above function to check missing data #if there was something missing , we should manupulate data to clean it

0.0 %


In [9]:
feature_names= diab_data.drop(['Outcome'],axis=1) #saving features data, removing Outcome column
x = feature_names

  #looking at features





In [10]:
#To find out columns having text or categorical values.
object_cols= [ col  for col, dt in x.dtypes.items() if dt == object]
object_cols

[]

In [11]:
# We import decision tree regressor
from sklearn.tree import DecisionTreeRegressor
diab_model = DecisionTreeRegressor(random_state=1)
#For model reproducibility, set a numeric value for random_state when specifying the model

In [12]:
# Fitting the model
diab_model.fit(x, y)

DecisionTreeRegressor(random_state=1)

In [13]:
#Predicting the values 
predictions = diab_model.predict(x)



In [14]:
#Validating our model(how accurate)
#here we are using Mean Absolute error for starters

from sklearn.metrics import mean_absolute_error

mean_absolute_error(y, predictions) # wowww, our model has 0 error , how come?coz we use the training data (x) to predict (y)!!

0.0

In [15]:
#Now we will split the data into train data, test data

from sklearn.model_selection import train_test_split  #import the library 


In [16]:
#splitting the data, random_state =0 makes sure we get same split every time
train_x, val_x, train_y, val_y = train_test_split(x, y, random_state = 0)

In [17]:
diab_model2 = DecisionTreeRegressor(random_state=1) #defining model again
diab_model.fit(train_x, train_y)                    #fitting the model with training data split


DecisionTreeRegressor(random_state=1)

In [18]:
#predicting the results using validation data split
val_predictions = diab_model.predict(val_x)


In [19]:
#Calculting error using validation outcome split, and validation split predictions
mean_absolute_error(val_y, val_predictions)

0.2708333333333333

In [20]:
#We use a utility function to help compare MAE scores from different values for max_leaf_nodes:

def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

In [21]:
# compare MAE with differing values of max_leaf_nodes using for loop
for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_x, val_x, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))

Max leaf nodes: 5  		 Mean Absolute Error:  0
Max leaf nodes: 50  		 Mean Absolute Error:  0
Max leaf nodes: 500  		 Mean Absolute Error:  0
Max leaf nodes: 5000  		 Mean Absolute Error:  0


In [22]:
#Using a different model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

forest_model = RandomForestRegressor(random_state=1)  #Using Random forest model
forest_model.fit(train_x, train_y)                    #fitting
melb_preds = forest_model.predict(val_x)              #predicting
print(mean_absolute_error(val_y, melb_preds))         #validating, finding MAE

0.2861458333333333


In [23]:
#Using pipelines
from xgboost import XGBRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer



In [24]:
model2= XGBRegressor(n_estimators=500, learning_rate=0.09)
model2.fit(train_x, train_y, verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.09, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=500, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [25]:
score= mean_absolute_error(model2.predict(val_x), val_y)
print(score)

0.29944806668875873


/opt/conda/lib/python3.7/site-packages/xgboost/data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +
